In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import re
import string
from tqdm import tqdm_notebook
from pandas_summary import DataFrameSummary
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, f1_score, auc as sk_auc, roc_curve, precision_score, recall_score

%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 30
pd.options.display.max_rows = 5000
seed = 42
np.random.seed(seed)

# Constants

In [2]:
# Target column name
TARGET_COLUMN = 'ProductCategoryLevel3UNSPSCCode'
data_path = '../data/'

SCENARIO2 = False
SCENARIO3 = True
GET_BUSINESS_CODE = True
if SCENARIO2 + SCENARIO3 != 1:
    raise Exception('Exactly one scenario should be selected')

# DIFI_CONTENT FILE: Catalog data

In [3]:
df_raw = pd.read_csv(data_path + 'DIFI_Content_201811030832.csv', sep=';', encoding='latin1')

C:\Users\BJOVAS\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1,3,7,8,10,12,13,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_raw.sample(20).loc[:, ['BuyerOrgLevel2', 'VendorName']]

,BuyerOrgLevel2,VendorName
2236163,Øyer kommune,Elkjøp Norge AS
3240452,Øyer kommune,BS Undervisning AS
2635137,Haugesund kommune,BS Undervisning AS
2756578,Randaberg kommune,BS Undervisning AS
3294323,Lillehammer kommune,BS Undervisning AS
687065,Ås kommune,BS Undervisning AS
2973571,Tysvær kommune,BS Undervisning AS
533654,Røyken kommune,BS Undervisning AS
4725162,Ås kommune,HEIDENREICH AS
2019680,Lillehammer kommune,Culina / Enor AS


In [5]:
sumdf = DataFrameSummary(df_raw).summary()
sumdf


,ProductCategoryLevel3UNSPSCCode,ProductCategoryLevel3Name,ProductCategoryLevel4UNSPSCCode,ProductCategoryLevel4Name,BuyerOrgLevel1,BuyerOrgLevel2,VendorName,VendorId,VendorTPID,CatalogueID,Contract,Product,VendorPartNumber,ClassificationCode,ClassificationName,UnitOfMeasure,ContentUnitsPerUnit,Manufacturer,ManufacturerProductNo,UnitPrice,Currency,CatalogLastModifiedYear
count,4.14819e+06,NaN,4.14819e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,4.90364e+07,NaN,4.90364e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.01929e+07,NaN,1.01929e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.01015e+07,NaN,1.01015e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,4.4102e+07,NaN,4.4102e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,5.51015e+07,NaN,5.51015e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,5.51015e+07,NaN,5.51015e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,9.51416e+07,NaN,9.51416e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
counts,4148190,4148190,4148190,4148190,5490807,5490807,5490807,4477613,3952174,5490807,1902992,5490782,5490807,411888,5490807,5470618,1553475,3725658,3263309,5490807,5490807,5356758
uniques,1073,1132,5060,5274,3,84,1049,1227,272,1709,616,1289932,1107843,153,154,59,14620,10720,550319,276991,2,1272


# Drop rows with missing code or description

In [6]:
df = df_raw.dropna(subset=[TARGET_COLUMN, 'Product'])
df.reset_index(inplace=True, drop=True)
df.shape

(4148165, 22)

## Drop rows where the product description is just a number

In [7]:
num_pattern = r'^[0-9]*((\.|\,)?[0-9]*)*$'
is_not_num = []
for product_name in df['Product'].values:
    m = re.match(num_pattern, product_name)
    is_not_num.append(not bool(m))
df = df.loc[is_not_num]
df.shape

(4148052, 22)

## Get level 3 code by integer division with 100. Drop unused columns

In [8]:
df['Label'] = df[TARGET_COLUMN].astype('int32')//100

In [9]:
df = df.loc[:, ['Product', 'Label', 'VendorName', 'CatalogueID']]
# Rename Product column to ProductName
df.columns = ['ProductName', 'Label', 'VendorName', 'CatalogueID']

In [10]:
df['ProductName'].value_counts()

Bibelen : Den heilage skrifta : Det gamle og Det nye testamentet Nynorsk Innbundet                                                                                  636
Bibelen : Guds ord Bokmål Innbundet                                                                                                                                 553
Bibelen : Den hellige skrift : Det gamle og Det nye testamentet Bokmål Innbundet                                                                                    526
Lise lyddempende bord 60x60cm, H:                                                                                                                                   414
Nora lyddempende bord 180x80cm, H:                                                                                                                                  414
Nora lyddempende bord 120x80cm, H:                                                                                                                              

## Drop duplicates

In [11]:
if SCENARIO2:
    df.drop_duplicates(inplace = True)
elif SCENARIO3:
    df.drop_duplicates(inplace = True, subset=['ProductName', 'Label'])
df.shape

(880036, 4)

In [12]:
df['ProductName'].unique().shape

(837526,)

# Get buisiness domain code

## Read data from Brønnøysund register

We only look at the primary business code as most businesses only deal with this

In [13]:
df_over = pd.read_csv(data_path + 'enheter_alle.csv', delimiter=';', encoding='latin1', dtype={'Næringskode 1': 'str'})
df_under = pd.read_csv(data_path + 'underenheter_alle.csv', delimiter=';', encoding='latin1', dtype={'Næringskode 1': 'str'})
df_over.dropna(subset=['Næringskode 1'], inplace=True)
df_under.dropna(subset=['Næringskode 1'], inplace=True)

C:\Users\BJOVAS\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\BJOVAS\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9,11,16,18,29,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Create new list of vendor names in  the catalog data and remove any trailing "AS" or "ASA"

In [14]:
def filter_name(name):
    split = name.split(' ')
    if split[-1] == 'as' or split[-1] == 'asa':
        return ''.join(split[:-1])
    return name

In [15]:
vendor_names_raw = df['VendorName'].unique()
names_raw = {}
vendor_names = []
for name in vendor_names_raw:
    vendor_name = filter_name(name.lower())
    vendor_names.append(vendor_name)
    names_raw[vendor_name] = name

## Start building a map for mapping raw vendor names as they appear in catalog data to business domain codes. Initial map is populated by manually found codes for some of the more frequent names for which we cannot automatically find a code

In [16]:
#No match
manually_mapped = {'ASKO/NorgesGruppen':'47111', 'LYRECO AS':'46660', 'TOOLS NORGE AS': '46900', 'ELEKTROSKANDIA AS':'46691', 'Staples Norway AS':'46499', 'Av Form AS':'46499', 'Plandent AS (tidligere NDD - AS Norsk Dental Depot':'46460', 'H. I. GIØRTZ SØNNER AS':'46390', 'Albert E. Olsen AS':'46630', 'Culina / Enor AS':'46441', 'Norgesgruppen (Asko)':'47111', 'Cater Drammen AS (a part of Servicegrossistene)':'46390', 'MÅKESTAD ENGROS ØST AS (A PART OF SERVICEGROSSISTENE)':'46390', 'Per Strand Tromsø AS':'46732','Telenor Inpli AS':'62030', 'Norengros Medisinske Forbruksvarer':'46460', 'AH Diagnostics':'46900', 'Odd Langdalen Frukt og Grønt Engros AS':'46310', 'E.A Smith AS':'46739', 'Nordialog':'46520', 'Atea\'\'AS':'46510', 'Tine BA':'10510', 'Tromsø Telecom AS':'71129', 'Radioservice Askøy AS':'47430', 'BIO-RAD LABORATORIES AB NORGE':'46750', 'Apro Tele og Data AS':'46520'}
#Match, but not a valid code
missing_codes = {'Atea AS':'46510', 'CREATIV COMPANY AS':'46499', 'Trigonor as':'46495', 'TINE SA':'10510', 'ITELLO AS':'46510'}
full_map = manually_mapped.copy()
full_map.update(missing_codes)

## Add the exact matches between catalog data and register data after applying the same AS/ASA filter to the names in the register data

We have access to both primary units and sub-units of businesses. The sub-unit register is only used if there is no match in the primary unit register

In [17]:
found = []
missing = []
over = 0
under = 0
over_names = dict([(filter_name(x.lower()), x) for x in df_over['Navn'].values])
under_names = dict([(filter_name(x.lower()), x) for x in df_under['Navn'].values])
for name in tqdm_notebook(vendor_names):
    raw_name = names_raw[name]
    if raw_name not in full_map:
        if name in over_names:
            found.append(name)
            code = df_over[df_over['Navn'] == over_names[name]].loc[:,['Næringskode 1']].values[0,0]
            full_map[raw_name] = ''.join(code.split('.'))
            over += 1
        else:
            if name in under_names:
                found.append(name)
                code = df_under[df_under['Navn'] == under_names[name]].loc[:, ['Næringskode 1']].values[0,0]
                full_map[raw_name] = ''.join(code.split('.'))
                under += 10
            else:
                missing.append(name)

In [18]:
len(full_map)

231

## Create maps for filtering out rows with identified business domain codes through vendor name. Extract subset of data

After adding the manually mapped businesses, we manage to map most of the businesses to codes. Those that are still missing are dropped. The alternative would be to keep them and say that they have unknown business codes, but as those with missing matches might have some correlation with eachother this might introduce bias to the model

In [19]:
mask = (df['VendorName'].isin(full_map))

In [20]:
df_filtered = df[mask]
df_miss = df[~mask]

In [21]:
df_filtered.shape, df.shape, df_filtered.shape[0]/df.shape[0]

((865161, 4), (880036, 4), 0.9830972823839025)

## Map vendor name to business code

In [22]:
df_filtered['Full_code'] = df_filtered['VendorName'].map(full_map)

C:\Users\BJOVAS\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Save to pickle

In [23]:
BUSINESS_NAME_ADDITION = ''
if GET_BUSINESS_CODE:
    df = df_filtered
    BUSINESS_NAME_ADDITION = '_business'

if SCENARIO2:
    df.to_pickle(data_path + 'catalog201810_scen2' + BUSINESS_NAME_ADDITION + '.pkl')
elif SCENARIO3:
    df.to_pickle(data_path + 'catalog201810_scen3' + BUSINESS_NAME_ADDITION + '.pkl')